<a href="https://colab.research.google.com/github/conchincradle/autoencoder_perceptual_loss/blob/main/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Horizon2333/imagenet-autoencoder

!pip install -r ./imagenet-autoencoder/requirements.txt

Cloning into 'imagenet-autoencoder'...
remote: Enumerating objects: 124, done.
remote: Total 124 (delta 0), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (124/124), 1.09 MiB | 23.80 MiB/s, done.
Resolving deltas: 100% (60/60), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 4.1 MB/s 


In [ ]:
!gdown --id 1WwJiQ1kBcNCZ37F6PJ_0bIL0ZeU3_sV8 --

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1WwJiQ1kBcNCZ37F6PJ_0bIL0ZeU3_sV8
To: /content/imagenet-vgg16.pth
100% 255M/255M [00:02<00:00, 127MB/s]


In [ ]:
import os
import shutil
shutil.move("/content/imagenet-vgg16.pth","/content/imagenet-autoencoder")

'/content/imagenet-autoencoder/imagenet-vgg16.pth'

In [ ]:
!ls
os.chdir("imagenet-autoencoder")
!ls

drive  imagenet-autoencoder  real1.jpg	sample_data
dataloader.py  imagenet-vgg16.pth  requirements.txt  train.py
eval.py        models		   run		     utils.py
figs	       README.md	   tools


In [ ]:
import torch
import torchvision
print(torch.cuda.is_available())


True


In [ ]:
# python tools/encode.py --arch {model architecture} --resume {checkpoint path} --img_path {image path}
# For example


!python tools/encode.py --arch vgg16 --resume imagenet-vgg16.pth --img_path figs/reconstruction.jpg

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 670, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 583, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 1043, in create_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
RuntimeError: KeyboardInterrupt: 

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "tools/encode.py", line 7, in <module>
    import torch
  File "/usr/local/lib/python3.7/dist-packages/torch/__init__.py", line 199, in <module>
    from torch._C import *  # noqa: F403
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
KeyboardInterrupt
^C


In [ ]:
#!/usr/bin/env python

import argparse

from PIL import Image

import torch
from torchvision.transforms import transforms

import sys
sys.path.append("./")

import utils
import models.builer as builder

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def get_args():
    # parse the args
    print('=> parse the args ...')
    parser = argparse.ArgumentParser(description='Encoder for auto encoder')
    parser.add_argument('--arch', default='vgg16', type=str, 
                        help='backbone architechture')
    parser.add_argument('--resume', type=str)
    parser.add_argument('--img_path',type=str)              
    
    args = parser.parse_args(["--arch","vgg16",'--resume',"imagenet-vgg16.pth",'--img_path',"figs/real1.jpg"])
    
    

    args.parallel = 0
    args.batch_size = 1
    args.workers = 0

    print(args)

    return args

def encode(model, img):

    with torch.no_grad():

        code = model.module.encoder(img).cpu().numpy()

    return code

def main(args):
    print('=> torch version : {}'.format(torch.__version__))

    utils.init_seeds(1, cuda_deterministic=False)

    print('=> modeling the network ...')
    model = builder.BuildAutoEncoder(args)     
    total_params = sum(p.numel() for p in model.parameters())
    print('=> num of params: {} ({}M)'.format(total_params, int(total_params * 4 / (1024*1024))))

    print('=> loading pth from {} ...'.format(args.resume))
    utils.load_dict(args.resume, model)
    
    trans = transforms.Compose([
                    transforms.Resize(256),                   
                    transforms.CenterCrop(224),
                    transforms.ToTensor()
                  ])

    img = Image.open(args.img_path).convert("RGB")

    img = trans(img).unsqueeze(0).cuda()

    model.eval()
    global code


    code = encode(model, img)
    print(code[0][0])
    

    # To do : any other postprocessing

if __name__ == '__main__':


    args = get_args()

    main(args)




=> parse the args ...
Namespace(arch='vgg16', batch_size=1, img_path='figs/real1.jpg', parallel=0, resume='imagenet-vgg16.pth', workers=0)
=> torch version : 1.11.0+cu113
=> modeling the network ...
=> num of params: 31888451 (121M)
=> loading pth from imagenet-vgg16.pth ...
[[0.10059284 0.08181164 0.05440877 0.04786503 0.1139784  0.09746966
  0.06345142]
 [0.05704408 0.06076542 0.05314697 0.0485983  0.08766764 0.07002585
  0.07253406]
 [0.05509628 0.06743823 0.05489914 0.07119349 0.09968959 0.06315568
  0.07919201]
 [0.07877008 0.09272821 0.0779055  0.06059049 0.09519731 0.07806238
  0.06707303]
 [0.09977935 0.06114728 0.06988992 0.04920468 0.10277557 0.0697617
  0.07723462]
 [0.09304125 0.06410365 0.06264398 0.05646518 0.08988865 0.04452971
  0.0815033 ]
 [0.06098513 0.05854816 0.04899107 0.04776639 0.09046818 0.05839682
  0.03120703]]


In [ ]:
import numpy as np
#wet2 = 2*(wet1-dry1)+code
3#newcode =code.copy()


3

In [ ]:
#!/usr/bin/env python

import argparse

import matplotlib.pyplot as plt

import torch

from torchvision.transforms import transforms

import sys
sys.path.append("./")

import utils
import models.builer as builder

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def get_args():
    # parse the args
    print('=> parse the args ...')
    parser = argparse.ArgumentParser(description='Trainer for auto encoder')
    parser.add_argument('--arch', default='vgg16', type=str, 
                        help='backbone architechture')
    parser.add_argument('--resume', type=str)      
    
    args = parser.parse_args(["--arch","vgg16",'--resume',"imagenet-vgg16.pth"])

    args.parallel = 0
    args.batch_size = 1
    args.workers = 0

    return args

def random_sample(arch):

    if arch in ["vgg11", "vgg13", "vgg16", "vgg19", "resnet18", "resnet34"]:
        return torch.randn((1,512,7,7))
    elif arch in ["resnet50", "resnet101", "resnet152"]:
        return torch.randn((1,2048,7,7))
    else:
        raise NotImplementedError("Do not have implemention except VGG and ResNet")

def main(args):
    print('=> torch version : {}'.format(torch.__version__))

    utils.init_seeds(1, cuda_deterministic=False)

    print('=> modeling the network ...')
    model = builder.BuildAutoEncoder(args)     
    total_params = sum(p.numel() for p in model.parameters())
    print('=> num of params: {} ({}M)'.format(total_params, int(total_params * 4 / (1024*1024))))

    print('=> loading pth from {} ...'.format(args.resume))
    utils.load_dict(args.resume, model)

    trans = transforms.ToPILImage()

   

    model.eval()
    print('=> Generating ...')
    with torch.no_grad():
        for i in range(512):
            newcode = code.copy()
            newcode += np.random.normal(0,0.001*i,512*7*7).reshape((512,7,7))
            #wet2 = 0.1*i*(wet1-dry1)+code
            input = torch.Tensor(newcode).cuda()
            

            output = model.module.decoder(input)
            
            output = trans(output.squeeze().cpu())
            filename  = "figs/0g"+str(i)+".jpg"
            output.save(filename)
    print("Completed")

            

   

if __name__ == '__main__':

    args = get_args()

    main(args)




=> parse the args ...
=> torch version : 1.11.0+cu113
=> modeling the network ...
=> num of params: 31888451 (121M)
=> loading pth from imagenet-vgg16.pth ...
=> Generating ...
Completed


In [ ]:
import os, tarfile
 
import os
from google.colab import files

def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
 
  files.download(output_filename)
 
 
make_targz_one_by_one('res.tar', '/content/imagenet-autoencoder/figs')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NameError: ignored